In [1]:
from os import listdir
from os.path import join
import csv

claims = []

with open('bases/allLupaFiles.tsv', 'r', encoding='utf-8') as f:
    read_claims = csv.DictReader(f, delimiter="\t", skipinitialspace=True)
    for claim in read_claims:
        claims.append(claim)


In [ ]:
import re

def endsWithQuotes(text):
    return text.endswith("″") or text.endswith("”") or text.endswith("“") or text.endswith("”.")

def starsWithQuotes(text):
    return text.startswith("“") or text.startswith("”")

def getParagraph(texts, initPos, endPos):
    paragraph = texts[initPos]
    for pos in range(initPos + 1, endPos + 1):
        paragraph += "\n" + texts[pos]
    return paragraph


def preprocessLupa(text):
    classes = ['VERDADEIRO, MAS', 'VERDADEIRO', 'EXAGERADO', 'FALSO', 'AINDA É CEDO PARA DIZER', 'CONTRADITÓRIO', 'SUBESTIMADO', 'INSUSTENTÁVEL', 'DE OLHO']    
    texts = text.split("\n")
    parts_to_ignore = ['Nota:', 'Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!',
        'O conteúdo produzido pela Lupa', 'Editado por:', 'Essa informação também foi verificada', 'Esta verificação foi sugerida por leitores', 
        'Uma versão semelhante dessa checagem foi feita', 'Nota da redação:']

    texts_treated = []
    for text in texts:
        ignore = False
        for part in parts_to_ignore:
            if part in text:
                ignore = True
                break
        if not ignore and text.strip() != '':
            texts_treated.append(text.strip())
    texts = texts_treated

    items = []

    for i in range(len(texts)):
        for classe in classes:
            if texts[i] == classe:                
                item = {'class': classe, 'metadata':'', 'classe_pos': i, 'trecho': len(items)}

                if endsWithQuotes(texts[i-1]):
                    endPos = i - 1
                    initPos = endPos
                    while not starsWithQuotes(texts[initPos]):
                        initPos -= 1                    
                    item['claim'] = getParagraph(texts, initPos, endPos)                    
                    item['claim_pos'] = initPos
                elif starsWithQuotes(texts[i-2]):
                    item['claim'] = texts[i-2]
                    item['claim_pos'] = i-2
                    item['metadata'] = texts[i-1]
                elif endsWithQuotes(texts[i-2]):
                    endPos = i - 2
                    initPos = endPos
                    while not starsWithQuotes(texts[initPos]):
                        initPos -= 1
                    item['claim'] = getParagraph(texts, initPos, endPos)                    
                    item['claim_pos'] = initPos
                    item['metadata'] = texts[i-1]
                elif texts[i-1].startswith('Checagem original') or texts[i-1].startswith('Texto em post'):
                    item['claim'] = texts[i-2]
                    item['claim_pos'] = i-2
                    item['metadata'] = texts[i-1]
                else:
                    item['claim'] = texts[i-1]
                    item['claim_pos'] = i-1
                item['claim'] = re.sub(r'^\d+. (.+)',r'\1', item['claim'])
                item['evidence'] = ''

                if len(items) > 0:
                    previousItem = items[-1]
                    initPosEvidence = previousItem['classe_pos'] + 1
                    endPosEvidence = item['claim_pos'] - 1
                    previousItem['evidence'] = getParagraph(texts, initPosEvidence, endPosEvidence)
                items.append(item)
                break
    for item in items:
        item['qtd_trechos'] = len(items)

    lastItem = items[-1]
    initPosEvidence = lastItem['classe_pos'] + 1
    endPosEvidence = len(texts) - 1
    lastItem['evidence'] = getParagraph(texts, initPosEvidence, endPosEvidence)

    return items

In [ ]:
from pathlib import Path

items = []

count = 0
for i, claim in enumerate(claims):
    items_claim = preprocessLupa(claim['content'])
    for item in items_claim:
        
        item['source'] = claim['url']
        item['row'] = claim['row']
        item['id'] = count
        if i == 433:
            print(item)
            print(len(items))
        items.append(item)            
        count += 1

In [4]:
text = """"Em julho de 2015, o Irã e um grupo de seis países nominado P5+1 (Estados Unidos, China, França, Rússia, Reino Unido e Alemanha), junto com a União Europeia, assinaram o Plano de Ação Conjunto Global (na sigla em inglês, JPCOA). Por esse acordo, o Irã se comprometia a reduzir sua capacidade de enriquecer urânio. Em troca, ativos iranianos que foram retidos por causa de sanções da ONU referentes à política nuclear do país seriam liberados.
O valor exato desses ativos é incerto. A estimativa mais alta é de que esses valores chegavam a US$ 150 bilhões. Entretanto, os valores estimados pelo próprio governo americano são bem menores. Segundo o Departamento do Tesouro dos Estados Unidos, o valor liberado é de “pouco mais de US$ 50 bilhões”. O acordo foi assinado durante a gestão de Barack Obama. Em 2018, Trump retirou a assinatura dos Estados Unidos.
Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!
É importante pontuar duas coisas: primeiro, isso não significa que esse dinheiro foi dado ao Irã. Esses ativos sempre foram do Irã, mas estavam congelados em bancos estrangeiros por causa de sanções impostas pela Organização das Nações Unidas (ONU). Além disso, praticamente todo o dinheiro estava em bancos fora dos Estados Unidos. O Politifact classificou essa informação como parcialmente correta."""

def preprocessClaim(text):
    text = text.replace("Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!", "")
    text = re.sub(r'^“',"", text)
    text = re.sub(r'”$',"", text)
    text = text.replace("_OPENQUOTES_","“")
    text = text.replace("_CLOSEQUOTES_","”")
    return text.strip()

def preprocessMetadata(text):
    text = text.replace("Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!", "")
    return text.strip()

def preprocessEvidence(text):
    text = text.replace("‌","")#Caractere estranho
    text = text.replace("Assine a Lente, a newsletter gratuita sobre desinformação da Lupa!", "")
    text = re.sub(r'Nota:.*',"", text)
    text = re.sub(r'Editado por:.*',"", text)
    text = re.sub(r'Esta verificação foi sugerida por leitores.*',"", text)
    return text.strip()

for item in items:
    item['claim'] = preprocessClaim(item['claim'])
    item['evidence'] = preprocessEvidence(item['evidence'])
    item['metadata'] = preprocessMetadata(item['metadata'])




In [5]:
#Salvar
with Path('bases/lupa_trechos_tratados.tsv').open('w', encoding="utf-8", newline='') as f:
    fieldnames = ['id', 'row', 'trecho', 'qtd_trechos', 'source', 'claim', 'metadata', 'class', 'evidence']
    print(fieldnames)
    writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='\t', extrasaction='ignore')
    writer.writeheader()
    for trecho in trechos:
        #print(trecho['evidence'])
        writer.writerow(trecho)

['id', 'row', 'trecho', 'qtd_trechos', 'source', 'claim', 'metadata', 'class', 'evidence']
